In [ ]:
!pip3 install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
###########________________MODIFIED_CODE______________________######################################
import os
import sklearn
from sklearn.model_selection import train_test_split

#Capture training data and labels into respective lists
images = []
labels = []

for directory_path in glob.glob("/content/drive/MyDrive/Speech-Data/mel_spectrogram_from_emotional_dataset/*"):
    data_split=os.path.split(directory_path)
    #label = directory_path.split("\\")[-1]
    label=data_split[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (256, 256))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        images.append(img)
        labels.append(label)
        
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, train_size = 0.7, random_state = 1, shuffle = True)

EMH_mel_spectrogram
EFS_mel_spectrogram
CMA_mel_spectrogram
CFS_mel_spectrogram


In [ ]:
#Convert lists to arrays        
train_images = np.array(X_train)
train_labels = np.array(Y_train)

In [ ]:
#Convert lists to arrays                
test_images = np.array(X_test)
test_labels = np.array(Y_test)


In [ ]:
test_labels

array(['CMA_mel_spectrogram', 'CMA_mel_spectrogram',
       'CFS_mel_spectrogram', 'CFS_mel_spectrogram',
       'EFS_mel_spectrogram', 'EMH_mel_spectrogram',
       'CMA_mel_spectrogram', 'CFS_mel_spectrogram',
       'EMH_mel_spectrogram', 'EFS_mel_spectrogram',
       'CMA_mel_spectrogram', 'CMA_mel_spectrogram',
       'EMH_mel_spectrogram', 'CFS_mel_spectrogram',
       'EFS_mel_spectrogram', 'EFS_mel_spectrogram',
       'EMH_mel_spectrogram', 'CFS_mel_spectrogram',
       'CFS_mel_spectrogram', 'CFS_mel_spectrogram',
       'EMH_mel_spectrogram', 'CFS_mel_spectrogram',
       'EFS_mel_spectrogram', 'CMA_mel_spectrogram',
       'EFS_mel_spectrogram', 'CFS_mel_spectrogram',
       'CMA_mel_spectrogram', 'EMH_mel_spectrogram',
       'EFS_mel_spectrogram', 'EFS_mel_spectrogram',
       'CMA_mel_spectrogram', 'CMA_mel_spectrogram',
       'EMH_mel_spectrogram', 'EFS_mel_spectrogram',
       'EMH_mel_spectrogram', 'EFS_mel_spectrogram',
       'EMH_mel_spectrogram', 'CFS_mel_spectro

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
test_labels[0:8]

array(['CMA_mel_spectrogram', 'CMA_mel_spectrogram',
       'CFS_mel_spectrogram', 'CFS_mel_spectrogram',
       'EFS_mel_spectrogram', 'EMH_mel_spectrogram',
       'CMA_mel_spectrogram', 'CFS_mel_spectrogram'], dtype='<U19')

In [ ]:
test_labels_encoded[0:8]

array([1, 1, 0, 0, 2, 3, 1, 0])

In [ ]:
#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded


In [ ]:
# Normalize pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
!pip3 install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#One hot encode y values for neural network. 
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
y_train_one_hot

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [ ]:
#Load model wothout classifier/fully connected layers
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False
    
VGG_model.summary()  #Trainable parameters will be 0

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
#Now, let us use features from convolutional network for RF
feature_extractor=VGG_model.predict(x_train)

In [ ]:
feature_extractor.shape

(420, 8, 8, 512)

In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
print(features.shape)

(420, 32768)


In [ ]:
X_for_RF = features #This is our X input to RF

In [ ]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators = 50, random_state = 42)

In [ ]:
# Train the model on training data
RF_model.fit(X_for_RF, y_train_one_hot) #For sklearn no one hot encoding

RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
print(RF_model.score(X_for_RF, y_train_one_hot))

1.0


In [ ]:
X_test_feature = VGG_model.predict(x_test)

In [ ]:

X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

In [ ]:
prediction_RF = RF_model.predict(X_test_features)
prediction_RF

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],


In [ ]:
#Print overall accuracy
from sklearn import metrics
print ("Accuracy = ", metrics.accuracy_score(y_test_one_hot, prediction_RF))

Accuracy =  0.9333333333333333
